In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/train']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

In [1]:
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
import numpy as np
# FIRST-AUTHOR: remove plotting
# import seaborn as sns
# import matplotlib.pyplot as plt

In [2]:
# FIRST-AUTHOR: remove ML code
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.svm import SVC

In [3]:
titanic = pd.read_csv('./input/train.scaled.csv')

In [4]:
titanic[['Age']].describe()

,Age
count,714.000000
mean,29.699118
std,14.526497
min,0.420000
25%,20.125000
50%,28.000000
75%,38.000000
max,80.000000


In [5]:
# FIRST-AUTHOR: remove plotting
# sns.heatmap(titanic.isnull(),yticklabels=False,cbar=False,cmap='viridis')
# plt.show()
_ = titanic.isnull()

In [6]:
# FIRST-AUTHOR: remove plotting
# sns.countplot(x='Survived',data=titanic,palette='RdBu_r');

In [7]:
# FIRST-AUTHOR: remove plotting
# sns.countplot(x='Survived',hue='Sex',data=titanic,palette='RdBu_r');

In [8]:
# FIRST-AUTHOR: remove plotting
# sns.countplot(x='Survived',hue='Pclass',data=titanic,palette='viridis');

In [9]:
# FIRST-AUTHOR: remove plotting
# sns.countplot(x='Pclass',hue='Survived',data=titanic,palette='viridis');

In [10]:
# FIRST-AUTHOR: remove plotting
# titanic[titanic.Survived == 0].hist("Age", bins=20)
# plt.show()
titanic[titanic.Survived == 0]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [11]:
titanic.Age = titanic.groupby('Pclass').Age.transform(lambda x: x.fillna(np.round(x.mean())))

In [12]:
# Extract titles from name
titanic['Title']=0
for i in titanic:
    titanic['Title']=titanic['Name'].str.extract('([A-Za-z]+)\.', expand=False)  # Use REGEX to define a search pattern

In [13]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15,5))
# sns.barplot(x=titanic['Title'], y=titanic['Age']);
# plt.show()
_ = titanic['Title']
_ = titanic['Age']

In [14]:
titanic_melted = pd.melt(titanic[['Survived', 'Age']], "Survived", var_name="Ages")

In [15]:
titanic_melted.head()

,Survived,Ages,value
0,0,Age,22.0
1,1,Age,38.0
2,1,Age,26.0
3,1,Age,35.0
4,0,Age,35.0


In [16]:
# FIRST-AUTHOR: remove plotting
# sns.swarmplot(x = 'Ages', y="value", hue="Survived", data=titanic_melted)
# plt.show()

In [17]:
df = titanic.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [18]:
sex_conv = lambda x : 1 if x=='male' else 0 
df['Is_male'] = df['Sex'].apply(sex_conv)

In [19]:
df['Age'] =pd.cut(df['Age'], bins=[1, 12, 50, 200], labels=['Child','Adult','Elder'])

In [20]:
embarked_new = pd.get_dummies(df['Embarked'])
train = pd.concat([df[['PassengerId','Survived','Pclass','Sex', 
                       'Age', 'SibSp', 'Parch', 'Fare','Embarked','Is_male']], embarked_new], axis=1)

In [21]:
# Family size feature
df['FamilySize'] = df['SibSp'] + df['Parch']
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,Title,Is_male,FamilySize
0,1,0,3,male,Adult,7.2500,S,Mr,1,1
1,2,1,1,female,Adult,71.2833,C,Mrs,0,1
2,3,1,3,female,Adult,7.9250,S,Miss,0,0
3,4,1,1,female,Adult,53.1000,S,Mrs,0,1
4,5,0,3,male,Adult,8.0500,S,Mr,1,0


In [22]:
titanic = pd.read_csv('./input/train.scaled.csv')
df = titanic.drop(['Ticket', 'Cabin'], axis=1)

#Converting Sex
sex_conv = lambda x : 1 if x=='male' else 0 
df['Is_male'] = df['Sex'].apply(sex_conv)

# Adding "Title" column
df['Title']=0
for i in df:
    df['Title']=df['Name'].str.extract('([A-Za-z]+)\.', expand=False)  

# Imputing Age  
df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(np.round(x.mean())))

# Converting Age
df['Age'] =pd.cut(df['Age'], bins=[1, 12, 50, 200], labels=['Child','Adult','Elder'])

# Adding Family column
df['FamilySize'] = df['SibSp'] + df['Parch']
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)

#Binilen limanları gruplandırma
embarked_new = pd.get_dummies(df['Embarked'])

train = pd.concat([df[['Survived','Pclass', 
                       'FamilySize','Fare',
                       'Is_male', 'Age']], embarked_new], axis=1)

X = train[['Is_male','Pclass', 'Age', "FamilySize"]]
y = train['Survived']